In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchsummary import summary
from tqdm.auto import tqdm
import cv2

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)

test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

In [3]:
class MNISTDataset(Dataset):
    def __init__(self,train = True):
        dataset_file_path = None
        if train:
            dataset_file_path = r"D:\MTech_IISc\OneDrive - Indian Institute of Science\Fourth Semester\UMCAIMLTA\PytorchTutorial\Dataset\data\mnist_train.csv"
        else:
            dataset_file_path = r"D:\MTech_IISc\OneDrive - Indian Institute of Science\Fourth Semester\UMCAIMLTA\PytorchTutorial\Dataset\data\mnist_test.csv"
        data = pd.read_csv(dataset_file_path)
        self.data_X = torch.empty(len(data),1,28,28)
        self.data_Y = torch.empty(len(data),dtype=torch.long)

        for data_index in range(len(data)):
            self.data_X[data_index] = torch.tensor(data.iloc[data_index,1:]).view(1,28,28)/255
            self.data_Y[data_index] = torch.tensor(data.iloc[data_index,0])

    def __getitem__(self,index):
        return self.data_X[index],self.data_Y[index]
    
    def __len__(self):
        return self.data_X.shape[0]

In [4]:
custom_train_data = MNISTDataset(train=True)

In [5]:
custom_test_data = MNISTDataset(train=False)

In [6]:
# Creating dataloaders from custom data
train_loader = DataLoader(custom_train_data, batch_size=16,shuffle=True)
test_loader = DataLoader(custom_test_data, batch_size=16,shuffle=True)

In [28]:
#Display one image
# image = train_data.data[0].unsqueeze(0).numpy()
image = custom_test_data[0][0].numpy()
cv2.imshow("MNISTImage",image.transpose(1,2,0))
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [7]:
# Creating dataloaders from torchvision dataset
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

In [8]:
class FFNN(nn.Module):
    def __init__(self):
        super(FFNN,self).__init__()

        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512,10)
        self.RELU = nn.ReLU()


    def forward(self,X):
        # X : [bs, 1, 28, 28]
        X = X.view(X.size(0),-1)

        X = self.fc1(X)
        X = self.RELU(X)
        X = self.fc2(X)

        return X
    

In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        self.RELU = nn.ReLU()

        self.fc = nn.Linear(32*5*5,10)

    def forward(self,X):
        X = self.conv1(X)
        X = self.max_pool(X)
        X = self.RELU(X)

        X = self.conv2(X)
        X = self.max_pool(X)
        X = self.RELU(X)

        X = X.view(X.size(0),-1)
        X = self.fc(X)

        return X

In [10]:
model = CNN()

In [11]:
summary(model,(1,28,28))

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 16, 26, 26]          416
├─MaxPool2d: 1-2                         [-1, 16, 13, 13]          --
├─ReLU: 1-3                              [-1, 16, 13, 13]          --
├─Conv2d: 1-4                            [-1, 32, 11, 11]          12,832
├─MaxPool2d: 1-5                         [-1, 32, 5, 5]            --
├─ReLU: 1-6                              [-1, 32, 5, 5]            --
├─Linear: 1-7                            [-1, 10]                  8,010
Total params: 21,258
Trainable params: 21,258
Non-trainable params: 0
Total mult-adds (M): 1.83
Input size (MB): 0.00
Forward/backward pass size (MB): 0.11
Params size (MB): 0.08
Estimated Total Size (MB): 0.20


Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 16, 26, 26]          416
├─MaxPool2d: 1-2                         [-1, 16, 13, 13]          --
├─ReLU: 1-3                              [-1, 16, 13, 13]          --
├─Conv2d: 1-4                            [-1, 32, 11, 11]          12,832
├─MaxPool2d: 1-5                         [-1, 32, 5, 5]            --
├─ReLU: 1-6                              [-1, 32, 5, 5]            --
├─Linear: 1-7                            [-1, 10]                  8,010
Total params: 21,258
Trainable params: 21,258
Non-trainable params: 0
Total mult-adds (M): 1.83
Input size (MB): 0.00
Forward/backward pass size (MB): 0.11
Params size (MB): 0.08
Estimated Total Size (MB): 0.20

In [12]:
criterion = nn.CrossEntropyLoss()

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [14]:
num_epochs = 2

In [15]:
for (images,labels) in train_loader:
    break

In [ ]:
#Training Loop

model.train()
training_steps = len(train_loader) * num_epochs
progress_bar = tqdm(range(training_steps))
step = 0

for epoch in range(num_epochs):
    for (images,labels) in train_loader:
        #Forward pass
        output_logits = model(images)

        #Compute loss
        loss = criterion(output_logits,labels)

        #Zero-out the grads
        optimizer.zero_grad()

        loss.backward()

        #updates weights
        optimizer.step()

        progress_bar.update(1)
        step+=1

        if step%100 == 0:
            print("Loss : ",loss.item())


In [17]:
def test_model(model_to_test):
    model_to_test.eval()
    correct = 0
    with torch.no_grad():
        for (images,labels) in test_loader:
            #Forward pass
            output = model_to_test(images)
            preds = torch.argmax(output,dim=-1)
            correct += (preds==labels).sum().item()
    return correct

In [18]:
correct = test_model(model)
correct

9614

In [19]:
Accuracy = correct/len(custom_test_data)
Accuracy

0.9614

In [20]:
path_to_save = r"D:\MTech_IISc\OneDrive - Indian Institute of Science\Fourth Semester\UMCAIMLTA\PytorchTutorial\Models\LinearModel1.pt"

In [23]:
torch.save(model.state_dict(),path_to_save)

In [24]:
New_Model = FFNN()

In [27]:
New_Model.load_state_dict(torch.load(path_to_save))

<All keys matched successfully>

In [28]:
correct = test_model(New_Model)

In [29]:
correct

9494